In [1]:
import os, glob, csv
from typing import List, Optional, Tuple
from model_evaluation import evaluate_model_on_collections
import numpy as np


In [2]:
def discover_models(models_dir: Optional[str], models_list: Optional[List[str]]) -> List[str]:
    paths = []
    if models_list:
        for p in models_list:
            if os.path.isfile(p) and p.endswith(".pt"):
                paths.append(p)
            else:
                print(f"[SKIP] Not a .pt file: {p}")
    if models_dir:
        for p in glob.glob(os.path.join(models_dir, "*.pt")):
            if os.path.isfile(p):
                paths.append(p)
    paths = sorted(list(dict.fromkeys(paths)))
    return paths

In [3]:
spaces = ["indoor","garage","outdoor"]
databases = ["wifi_fingerprinting_data_raw","wifi_fingerprinting_data","wifi_fingerprinting_data_exponential","wifi_fingerprinting_data_extra_features_no_leak"]
BATCH_SIZE = 2048

In [5]:
from feature_lists import DATASET_TO_FEATURE
MODELS_DIR = "retrained_models"
models = []

models = discover_models(MODELS_DIR, models)


rows = []
for mp in models:

    model_path = mp.split("/")[1].replace(".pt","").split("_")
    relative_coeficient = 32

    model_type = model_path.pop(0)
    space = max((s for s in spaces if s in mp), key=len, default=None)
    database = max((d for d in databases if d in mp), key=len, default=None)
    feature_list  = DATASET_TO_FEATURE[database]
    cols = [f"reto_grande_{space}"]

    if database == "wifi_fingerprinting_data_raw":
        relative_coeficient = 3.2

    mae, mse, rmse, n = evaluate_model_on_collections(
        mp, cols, database, BATCH_SIZE,None,feature_list
    )
    mae_final = mae * relative_coeficient
    mse_final = mse * relative_coeficient
    rmse_final = rmse * relative_coeficient
    print(f"{os.path.basename(mp)} | N={n} | MAE={mae_final:.6f} | MSE={mse_final:.6f} | RMSE={rmse_final:.6f}")
    rows.append((os.path.basename(mp), mae_final, mse_final, rmse_final))



CNN_garage_wifi_fingerprinting_data.pt | N=23160 | MAE=2.713941 | MSE=0.498745 | RMSE=3.994977
CNN_garage_wifi_fingerprinting_data_exponential.pt | N=23160 | MAE=4.068295 | MSE=0.897750 | RMSE=5.359850
CNN_garage_wifi_fingerprinting_data_extra_features_no_leak.pt | N=22621 | MAE=11.367534 | MSE=5.889815 | RMSE=13.728587
CNN_garage_wifi_fingerprinting_data_raw.pt | N=23160 | MAE=2.978427 | MSE=5.729020 | RMSE=4.281690
CNN_indoor_wifi_fingerprinting_data.pt | N=12243 | MAE=2.775443 | MSE=0.551603 | RMSE=4.201345
CNN_indoor_wifi_fingerprinting_data_exponential.pt | N=12243 | MAE=1.868392 | MSE=0.233014 | RMSE=2.730652
CNN_indoor_wifi_fingerprinting_data_extra_features_no_leak.pt | N=12243 | MAE=0.055096 | MSE=0.004409 | RMSE=0.375630
CNN_indoor_wifi_fingerprinting_data_raw.pt | N=12243 | MAE=1.376466 | MSE=1.500267 | RMSE=2.191085
CNN_outdoor_wifi_fingerprinting_data.pt | N=19910 | MAE=3.040233 | MSE=0.597947 | RMSE=4.374276
CNN_outdoor_wifi_fingerprinting_data_exponential.pt | N=19910 | 

In [6]:
import pandas as pd

df = pd.DataFrame(rows, columns=["model", "mae", "mse", "rmse"])

styled = df.style.background_gradient(cmap="RdYlGn_r")  # _r reverses so green=low, red=high
styled

,model,mae,mse,rmse
0,CNN_garage_wifi_fingerprinting_data.pt,2.713941,0.498745,3.994977
1,CNN_garage_wifi_fingerprinting_data_exponential.pt,4.068295,0.897750,5.359850
2,CNN_garage_wifi_fingerprinting_data_extra_features_no_leak.pt,11.367534,5.889815,13.728587
3,CNN_garage_wifi_fingerprinting_data_raw.pt,2.978427,5.729020,4.281690
4,CNN_indoor_wifi_fingerprinting_data.pt,2.775443,0.551603,4.201345
5,CNN_indoor_wifi_fingerprinting_data_exponential.pt,1.868392,0.233014,2.730652
6,CNN_indoor_wifi_fingerprinting_data_extra_features_no_leak.pt,0.055096,0.004409,0.375630
7,CNN_indoor_wifi_fingerprinting_data_raw.pt,1.376466,1.500267,2.191085
8,CNN_outdoor_wifi_fingerprinting_data.pt,3.040233,0.597947,4.374276
9,CNN_outdoor_wifi_fingerprinting_data_exponential.pt,4.299556,1.055789,5.812509


In [ ]:
MODELS_DIR = "retrained_cnn"
models = []

models = discover_models(MODELS_DIR, models)


rows = []
for mp in models:

    model_path = mp.split("/")[1].replace(".pt","").split("_")
    relative_coeficient = 32

    model_type = model_path.pop(0)
    space = max((s for s in spaces if s in mp), key=len, default=None)
    database = max((d for d in databases if d in mp), key=len, default=None)
    cols = [f"reto_grande_{space}"]

    print(f"type {model_type}, space {space}, database {database}, cols {cols}")

    if database == "wifi_fingerprinting_data_raw":
        relative_coeficient = 1

    mae, mse, rmse, n = evaluate_model_on_collections(
        mp, cols, database, BATCH_SIZE
    )
    mae_final = mae * relative_coeficient
    mse_final = mse * relative_coeficient
    rmse_final = rmse * relative_coeficient
    print(f"{os.path.basename(mp)} | N={n} | MAE={mae_final:.6f} | MSE={mse_final:.6f} | RMSE={rmse_final:.6f}")
    rows.append((os.path.basename(mp), mae_final, mse_final, rmse_final))



type CNN, space garage, database wifi_fingerprinting_data_extra_features_no_leak, cols ['reto_grande_garage']


CNN_EXTRA_FEAT_FINAL_garage_wifi_fingerprinting_data_extra_features_no_leak.pt | N=22621 | MAE=3.084549 | MSE=0.576184 | RMSE=4.293936
type CNN, space indoor, database wifi_fingerprinting_data_extra_features_no_leak, cols ['reto_grande_indoor']
CNN_EXTRA_FEAT_FINAL_indoor_wifi_fingerprinting_data_extra_features_no_leak.pt | N=12243 | MAE=0.103684 | MSE=0.000392 | RMSE=0.112046
type CNN, space outdoor, database wifi_fingerprinting_data_extra_features_no_leak, cols ['reto_grande_outdoor']
CNN_EXTRA_FEAT_FINAL_outdoor_wifi_fingerprinting_data_extra_features_no_leak.pt | N=19550 | MAE=5.994944 | MSE=1.672754 | RMSE=7.316293
type CNN, space garage, database wifi_fingerprinting_data_exponential, cols ['reto_grande_garage']
CNN_XY_RSSI_norm_FINAL_garage_wifi_fingerprinting_data_exponential.pt | N=23160 | MAE=4.265374 | MSE=0.964118 | RMSE=5.554437
type CNN, space indoor, database wifi_fingerprinting_data_exponential, cols ['reto_grande_indoor']
CNN_XY_RSSI_norm_FINAL_indoor_wifi_fingerprintin

In [ ]:
import pandas as pd

df = pd.DataFrame(rows, columns=["model", "mae", "mse", "rmse"])

styled = df.style.background_gradient(cmap="RdYlGn_r")  # _r reverses so green=low, red=high
styled

,model,mae,mse,rmse
0,CNN_EXTRA_FEAT_FINAL_garage_wifi_fingerprinting_data_extra_features_no_leak.pt,3.084549,0.576184,4.293936
1,CNN_EXTRA_FEAT_FINAL_indoor_wifi_fingerprinting_data_extra_features_no_leak.pt,0.103684,0.000392,0.112046
2,CNN_EXTRA_FEAT_FINAL_outdoor_wifi_fingerprinting_data_extra_features_no_leak.pt,5.994944,1.672754,7.316293
3,CNN_XY_RSSI_norm_FINAL_garage_wifi_fingerprinting_data_exponential.pt,4.265374,0.964118,5.554437
4,CNN_XY_RSSI_norm_FINAL_indoor_wifi_fingerprinting_data_exponential.pt,2.117519,0.261938,2.895170
5,CNN_XY_RSSI_norm_FINAL_outdoor_wifi_fingerprinting_data_exponential.pt,4.316021,1.066526,5.841989
6,CNN_XY_norm_FINAL_garage_wifi_fingerprinting_data.pt,2.922153,0.539197,4.153829
7,CNN_XY_norm_FINAL_indoor_wifi_fingerprinting_data.pt,2.740024,0.549681,4.194019
8,CNN_XY_norm_FINAL_outdoor_wifi_fingerprinting_data.pt,3.103208,0.621058,4.458011


In [ ]:
DELTA_ROOT = "/home/admindi/sbenites/WirelessLocation/data_processing/hyperparameter_tunning/CNN_hyperparameter_optimization/model_storage_DELTA_FINAL_outdoor"
mp = f"{DELTA_ROOT}/all_data_run0_depth3_model4.pt"
col = ["reto_grande_outdoor"]
database = "wifi_fingerprinting_data_extra_features_no_leak"
features = [
    "delta_freind1_rssi_freind2_rssi",
    "delta_freind2_rssi_freind3_rssi",
    "delta_freind3_rssi_freind1_rssi",
]
relative_coeficient = 32
mae, mse, rmse, n = evaluate_model_on_collections(mp, cols, database, BATCH_SIZE,None,features)
mae_final = mae * relative_coeficient
mse_final = mse * relative_coeficient
rmse_final = rmse * relative_coeficient
print(f"{os.path.basename(mp)} | N={n} | MAE={mae_final:.6f} | MSE={mse_final:.6f} | RMSE={rmse_final:.6f}")


all_data_run2_depth8_model0.pt | N=19550 | MAE=4.226872 | MSE=1.032029 | RMSE=5.746732
